<a href="https://colab.research.google.com/github/RafaelSJ0/Aulas-Mack/blob/main/Trilha_6_An%C3%A1lise_de_Sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregar as bibliotecas e dados

## Bibliotecas

In [ ]:
!pip install unidecode
!pip install fractions
!pip install googletrans
!pip install ibm_watson
!pip install ibm_cloud_sdk_core
!pip install dotenv


     |████████████████████████████████| 235 kB 19.5 MB/s 
ERROR: Could not find a version that satisfies the requirement fractions (from versions: none)
ERROR: No matching distribution found for fractions
     |████████████████████████████████| 55 kB 1.5 MB/s 
     |████████████████████████████████| 1.3 MB 10.8 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 53 kB 1.2 MB/s 
     |████████████████████████████████| 65 kB 1.6 MB/s 
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15735 sha256=978290d00c2185a3fa6ee55c0fa3acd5bc7b2c0e81d10cb719a650936f5ad77d
  Stored in directory: /root/.cache/pip/wheels/20/da/eb/a54579056f265eede0417df537dd56d3df5b9eb2b25df0003d
Successfully built googletrans
     |████████████████████████████████| 338 kB 19.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |█████████████████████████

In [ ]:
# Load EDA Pkgs
import pandas as pd
import numpy as np
import random

# Load ML Pkgs
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('rslp')
from textblob import TextBlob

#charts & others stuff
from unidecode import unidecode
import matplotlib.pyplot as plt
import re
import string
from fractions import Fraction
import json
import os
from googletrans import Translator
translator = Translator()


# from dotenv import load_dotenv
# load_dotenv()
# load_dotenv('../ibm-credentials.env')

# IBM Watson
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson import LanguageTranslatorV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1  import Features, CategoriesOptions, SentimentOptions



# NATURAL_LANGUAGE_UNDERSTANDING_APIKEY = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_APIKEY']
# NATURAL_LANGUAGE_UNDERSTANDING_URL    = os.environ['NATURAL_LANGUAGE_UNDERSTANDING_URL']

# LANGUAGE_TRANSLATOR_APIKEY = os.environ['LANGUAGE_TRANSLATOR_APIKEY']
# LANGUAGE_TRANSLATOR_URL    = os.environ['LANGUAGE_TRANSLATOR_URL']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.


In [ ]:
authenticator = IAMAuthenticator(f'{NATURAL_LANGUAGE_UNDERSTANDING_APIKEY}')
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2021-08-01',
    authenticator=authenticator
)

natural_language_understanding.set_service_url(f'{NATURAL_LANGUAGE_UNDERSTANDING_URL}')

NameError: ignored

## Carrega os Tweets

In [ ]:
# #Carrega todos os arquivos
# df_o = pd.DataFrame()
# for filename in os.listdir('../data/'):    
#     df_o = df_o.append(pd.read_json(f"../data/{filename}", lines=True))

# #remove tweets que eventualmente estao duplicados 
# df_o.drop_duplicates(subset=['id_str', 'search_term'], inplace=True)

# #Recria o indice, pq carregou por pedacos
# df_o.reset_index(drop=True, inplace=True)
# df_o.shape

### Restaura BAckup caso necessario (evitar gastar a conta da IBM Cloud)

In [ ]:
# df = pd.read_csv(f"tweets_com_ibm_watson_old.csv")
df_o = pd.read_csv(f"tweets_com_ibm_watson_e_ingles.csv")

# df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)


### FAz uma proxy, just because

In [ ]:
#df = df_o[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source']]
df = df_o# .sample(20)
df.shape

(9673, 24)

# Marca o sentimento a partir do IBM Watson

## Teste do IBM Watson

In [ ]:
#response = natural_language_understanding.analyze(language='pt', text='inclusive muito feliz que entrou pro catálogo da amazon prime', features=Features(sentiment=SentimentOptions())).get_result()
# print(json.dumps(response, indent=2))
''' 
{
  "usage": {
    "text_units": 1,
    "text_characters": 61,
    "features": 1
  },
  "sentiment": {
    "document": {
      "score": 0.862283,
      "label": "positive"
    }
  },
  "language": "pt"
}
'''
# response = natural_language_understanding.analyze(language='pt', text='Que coisa chata essa transmissão do #Bellator268 no Star+. Torcida gratuita com narrador e comentarista acabando co… https://t.co/2y7Y0cJO0e', features=Features(sentiment=SentimentOptions())).get_result()
# print(json.dumps(response, indent=2))
''' 
{
  "usage": {
    "text_units": 1,
    "text_characters": 140,
    "features": 1
  },
  "sentiment": {
    "document": {
      "score": -0.748979,
      "label": "negative"
    }
  },
  "language": "pt"
}
'''


## Marca a base de dados com o IBM Watson

In [ ]:
# Conecta nos servidores do Watson e realiza a analise de sentimento
# guarda o resultado em uma coluna
def WatsonSentiment(Tweet):
    response = natural_language_understanding.analyze(language='pt', text=Tweet, features=Features(sentiment=SentimentOptions())).get_result()
    #print(f"{response['sentiment']['document']['score']} : {Tweet}")
    return response['sentiment']['document']['score'], response['sentiment']['document']['label']

# Cria as novas colunas do DataFrame
df['ibm_watson_sentiment'], df['ibm_watson_sentiment_label'] = zip(*df['text'].map(WatsonSentiment))

In [ ]:
#df[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source', 'ibm_watson_sentiment', 'ibm_watson_sentiment_label']].sample(10)
df[['id_str', 'text', 'screen_name', 'search_term', 'search_term_group', 'source']].loc[df['search_term_group'].ne('star+')].sample(15)

## Salva para nao perder

In [ ]:
filename = "tweets_com_ibm_watson"
df.to_csv(f"{filename}.csv")
df.to_json(f"{filename}.json", orient='values')
df.to_excel(f"{filename}.xlsx")

In [ ]:
df.sample(20)

# Prepara a base para rodar com a versão em ingles do TextBlob

## Teste do Google Translator

In [ ]:
response = translator.translate('inclusive muito feliz que entrou pro catálogo da amazon prime', src="PT", dest='EN').text
print(json.dumps(response, indent=2))

## Testa o IBM Watson Translator

In [ ]:
# Autenticação com a chave gerada na IBM CLoud
authenticator = IAMAuthenticator(f'{LANGUAGE_TRANSLATOR_APIKEY}')

#Inicia a componente com a URL provida na IBM Cloud
language_translator = LanguageTranslatorV3(
    version='2018-05-01',
    authenticator=authenticator
)
language_translator.set_service_url(f'{LANGUAGE_TRANSLATOR_URL}')

#Testa uma tradução (cuidado com a quantidade de eventos gratuitos)
# translation = language_translator.translate(
#     text='Hello, how are you today?',
#     model_id='en-pt').get_result()

# Imprime o resultado
# print(json.dumps(translation, indent=2, ensure_ascii=False))
''' 
{
  "translations": [
    {
      "translation": "Olá, como você está hoje?"
    }
  ],
  "word_count": 7,
  "character_count": 25
}
'''

## Marca a versao Inglesa via TextBlob

### Conta os caracteres para avaliar junto ao servico gratuito se cabe

In [ ]:
len("".join(df['text']))


dft = df.sample(3)
dft

### Primeiro, traduz o texto

In [ ]:
# Traduzindo para o Inglês
# Conecta nos servidores do Watson e realiza a analise de sentimento
# guarda o resultado em uma coluna
def WatsonTranslate(Tweet):
    response = language_translator.translate(text=Tweet,  model_id='pt-en').get_result()
    translations = response['translations']    
    #print(f"{translations[0]['translation']} : {Tweet} :: {response['character_count']}")
    return translations[0]['translation'], response['character_count']


df['textEN'], df['ibm_watson_tranlation_character_count'] = zip(*df['text'].map(WatsonTranslate))
#dft['textEN'], dft['ibm_watson_tranlation_character_count'] = zip(*dft['text'].map(WatsonTranslate))

# Versão Google (Rate Limit applied)
#df['textEN'] = df['text'].apply(lambda x: translator.translate(x, src="PT", dest='EN').text)

In [ ]:
df

### Marca sentimento pelo TextBlob

In [ ]:
#Calculando o sentimento...
df['textblob_polarity']     = df['textEN'].apply(lambda x: TextBlob(x).sentiment.polarity)
df['textblob_subjectivity'] = df['textEN'].apply(lambda x: TextBlob(x).sentiment.subjectivity)

## Salva para nao perder

In [ ]:
filename = "tweets_com_ibm_watson_e_ingles"
df.to_csv(f"{filename}.csv")
df.to_json(f"{filename}.json", orient='values')
df.to_excel(f"{filename}.xlsx")

# Naive Bayes

In [ ]:
df_nb = df.sample(10)
vectorizer = CountVectorizer()

#Arnaldo Jabor
X = vectorizer.fit_transform(df_nb['text'])
Y = df_nb['ibm_watson_sentiment_label']
vocabulary = vectorizer.get_feature_names()
dfX = pd.DataFrame(data=X.toarray(), columns=vocabulary) #.iloc[:,0::2]
Y

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


7875     neutral
9482    positive
933     negative
2031    negative
3960    negative
1487    negative
1371    positive
7795    negative
3671    positive
7438     neutral
Name: ibm_watson_sentiment_label, dtype: object

In [ ]:
df.describe()

,Unnamed: 0,favorite_count,followers_count,id,id_str,retweet_count,ibm_watson_sentiment,ibm_watson_tranlation_character_count,textblob_polarity,textblob_subjectivity
count,9673.000000,9673.000000,9.673000e+03,9.673000e+03,9.673000e+03,9673.000000,9673.000000,9673.000000,9673.000000,9673.000000
mean,4836.000000,11.359971,5.691394e+04,4.832581e+17,1.450957e+18,0.893725,0.031843,101.745270,0.080189,0.335639
std,2792.498911,167.273884,4.698256e+05,5.701233e+17,8.789754e+14,17.747758,0.666091,35.971436,0.294877,0.325208
min,0.000000,0.000000,0.000000e+00,7.927950e+05,1.449260e+18,0.000000,-0.997954,12.000000,-1.000000,0.000000
25%,2418.000000,0.000000,1.290000e+02,1.064530e+08,1.450251e+18,0.000000,-0.640356,70.000000,0.000000,0.000000
50%,4836.000000,1.000000,4.350000e+02,2.948113e+09,1.450976e+18,0.000000,0.000000,106.000000,0.000000,0.316667
75%,7254.000000,2.000000,2.026000e+03,1.077593e+18,1.451619e+18,0.000000,0.691498,140.000000,0.200000,0.600000
max,9672.000000,13936.000000,1.696435e+07,1.452034e+18,1.452619e+18,1227.000000,0.999999,180.000000,1.000000,1.000000
